# LLM Prompt Engineering Lab
## Tree of Thought Prompting with GPT4o-mini

Preliminaries: import libraries, read in the OpenAI API key and pass it to an OpenAI object constructor

In [ ]:
!pip install anytree
!pip install openai
!pip install python-dotenv

**TOKEN_TOTALS Dictionary:** Dictionary to store total token counts

In [ ]:
TOKEN_TOTALS = {"prompt_tokens": 0, "completion_tokens": 0, "total_tokens": 0}

def reset_token_totals():
    # Call this after solving each maze to reset it to 0.
    TOKEN_TOTALS["prompt_tokens"] = 0
    TOKEN_TOTALS["completion_tokens"] = 0
    TOKEN_TOTALS["total_tokens"] = 0

In [ ]:
# Import libraries, set API key
from anytree import Node, RenderTree

import ASCIImaze
import os
import re

# This chunk of code reads an OpenAI API key from a chosen environment variable, which must be predefined.
# For my own dev, I'm using my personal API key. This will not be provided to others, you must use one you have
# access to.
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

from openai import OpenAI

client = OpenAI(
    api_key=os.environ['OPENAI_D31_747393_KEY']
)


<b>First prompt:</b> Recognize the ASCII maze, and format it as a single-line string, with the different lines of the maze delimited by "|"

In [ ]:
extract_maze_system_prompt = """
INPUT
- One fenced code block that contains ONLY the raw ASCII maze grid.
- The allowed characters are '#', 'S', 'E', '*', and space.
- Be aware that some rows might have leading or trailing spaces. Do NOT trim them.

TASK
- Convert the multi-line grid into a single line by replacing each newline between rows with exactly one '|' character.
- Keep every other character exactly as it is given even including the spaces.

RULES (STRICT)
1) Output exactly one line and nothing else.
2) Do NOT add a leading or trailing '|'.
3) Do NOT add any text, quotes, labels, or backticks.
4) Do NOT remove or insert spaces.
5) Keep each row byte-for-byte the same.
6) Keep the original row widths and if a row ends with spaces, keep those spaces.

OUTPUT
- Return only the single reformatted line.

EXAMPLE
Input:
###
#S#
# #
#E#
###
Output:
###|#S#|# #|#E#|###
"""

<b>Second prompt:</b> Format the 3x3 neighborhood in the same manner as the ASCII maze.

In [ ]:
generate_thought_context_system_prompt = """
INPUT
- A single fenced code block that includes a 3x3 ASCII grid centered on '*'.
- Exactly three grid lines exist for the neighborhood and each line is exactly 3 characters wide.
- Allowed characters in the grid are '#', 'S', 'E', '*', and space.
- Ignore any non-grid lines (e.g., headings like 'Here is a 3x3...' or prompts like 'Enter a move...').

TASK
- Extract only the three 3-character grid lines.
- Join them into one line by placing a single '|' between each row: <row1>|<row2>|<row3>.

RULES (STRICT)
1) The final output MUST exactly be 11 characters and not less and not more: 3 + '|' + 3 + '|' + 3. Count them before answering.
2) Preserve every character exactly (including spaces). Do not trim.
3) If the row length is less than that of 3 characters pad the sides with a space where necessary.
4) Do not add any extra text, quotes, labels, or backticks.
5) Do NOT add a leading or trailing '|'.
6) Only the three grid rows may appear in the output (joined with '|').

OUTPUT
- Return only the single 11-character string and of form <row1>|<row2>|<row3>.

EXAMPLE 1
Input:
# #
#*#
##E
Output:
# #|#*#|##E

EXAMPLE 2
Input:
###
 *
 ##
Output:
###| * | ##

EXAMPLE 3
Input:
##
 *
###
Output:
## | * |###

EXAMPLE 4
Input:
###
 *
##
Output:
###| * |##
"""

<b>Third prompt:</b> Based on the refomatted ASCII maze and neighborhood from prompts 1 and 2 above, choose the next move to make.

In [ ]:
generate_thought_system_prompt = """
INPUT
- The full ASCII maze flattened with '|' between rows,
- The current 3×3 neighborhood flattened as <row1>|<row2>|<row3>.
- You will also be told:
  - ALLOWED MOVES for this turn: {vmove}
  - BACKTRACK direction to avoid unless it is the only remaining option: {backmove}

TASK
- Choose exactly one next move that progresses toward 'E' using a deterministic DFS policy.

DECISION POLICY (DETERMINISTIC DFS)
1) Start with the candidate set = ALLOWED MOVES for this turn.
2) If the backtrack direction is in the candidate set and the set has more than one option, remove the backtrack.
3) Apply this fixed priority order and pick the first option that appears in the candidate set: U > R > L > D.
4) If nothing remains after step 2, choose the backtrack direction (it is the only possible option).

RULES (STRICT)
- Never choose a move that is not in the allowed set for this turn.
- Final output must be ONE uppercase letter in [U, D, L, R].
- Do not include any other characters such as spaces, punctuation, extra words, labels, or backticks.

OUTPUT
- Return only the single move letter (U, D, L, or R).

EXAMPLE 1
Allowed moves: U, R, D
Backtrack: L
Output: U

EXAMPLE 2
Allowed moves: L
Backtrack: L
Output: L
"""


In [ ]:
# Possible models:
# gpt-3.5-turbo
# gpt-4o-mini

# get_completion is a helper method suggested in the DeepLearning.ai course on prompt engineering.
# I modified this version to use the OpenAI 1.0 implementation. The "client" object was passed an API key in line 24
def get_completion(system_prompt, user_prompt, my_model="gpt-4o-mini"):
    completion = client.chat.completions.create(
        model=my_model,
        messages=[
            {"role": "system",
             "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )
    # getting the token counts per call
    prompt_tokens = completion.usage.prompt_tokens
    completion_tokens = completion.usage.completion_tokens
    total_tokens = completion.usage.total_tokens

    # adding the token counts per call to get total counts per maze
    TOKEN_TOTALS["prompt_tokens"] += prompt_tokens
    TOKEN_TOTALS["completion_tokens"] += completion_tokens
    TOKEN_TOTALS["total_tokens"] += total_tokens

    return completion.choices[0].message


# -------------------------------------------------------------------------------------------------------------

# Utility function for displaying trees. Core code from the anytree package documentation.
def render_that_tree(mytree):
    for pre, _, node in RenderTree(mytree):
        print("%s%s" % (pre, node.name))

# ---------------------------------------------------------------------------------------------------------------

# Backtracking subroutine for Tree of Thought. Updates the parent's state info to prune the dead-end branch we are
#  backtracking from (by setting that position in the stored 3x3 to '#').

def tree_backtrack(deadend_node, curr_move):

    curr_node = deadend_node.parent

    if deadend_node.move == 'D':
        temp = list(curr_node.name)
        temp[9] = '#'
        curr_node.name = ''.join(temp)

    elif deadend_node.move == 'U':
        temp = list(curr_node.name)
        temp[1] = '#'
        curr_node.name = ''.join(temp)

    elif deadend_node.move == 'L':
        temp = list(curr_node.name)
        temp[4] = '#'
        curr_node.name = ''.join(temp)

    elif deadend_node.move == 'R':
        temp = list(curr_node.name)
        temp[6] = '#'
        curr_node.name = ''.join(temp)

    else:
        print("Illegal move in backtracking")
        return None

    return curr_node

# ----------------------------------------------------------------------------------------------------------------

# Add node subroutine for Tree of Thought. Pretty short, but putting it here makes the main loop cleaner.
#  This also pushes the tree updates out to subroutines, which is common and familiar. Leaves just the LLM and game
#  logic in the main loop, which I think is good.
def add_node(curr_node, LLM_move_cleaned):
    # New node is a child of the current node. State has to be determined on next iteration. The move to reach this
    #  node from its parent is recorded as the "move" attribute. The move to reach its parent from this node
    #  is recorded as the "backtrack" attribute.

    # Have to determine the backtrack attribute by reversing LLM_cleaned_move. Another elif tree here.
    if LLM_move_cleaned.find('D') >= 0:
        rev = 'U'

    elif LLM_move_cleaned.find('U') >= 0:
        rev = 'D'

    elif LLM_move_cleaned.find('L') >= 0:
        rev = 'R'

    elif LLM_move_cleaned.find('R') >= 0:
        rev = 'L'

    else:
        print("Not a legal move")
        return None

    temp = Node(parent=curr_node, name="", move=LLM_move_cleaned, backtrack=rev)

    return temp


# ----------------------------------------------------------------------------------------------------------

def evaluate_thought(curr_local, curr_node):
    # Pre: curr_local is the 3x3 neighborhood for the current location in the maze.
    # Format ###|#*#|###
    # curr_node is the current node of the tree, the center point of the 3x3 local neighborhood.
    # Post: none
    # Returns: List of valid moves, each a 1-character string from {U, D, L, R}

    # Create output list, convert input string to a list.

    # Debug
    print(f"evaluate thought recevied {curr_local}")
    # Debug

    valid_moves = []
    maze_neigh = list(curr_local)
    # print(f"Local neighborhood string is {curr_local}")
    # print(f"Converted to a List it is {maze_neigh}")

    # Checking validity of each possible move
    # BUG TO SQUASH: Must not add the backtrack direction unless it's the only open move.
    if maze_neigh[1] == ' ' or maze_neigh[1] == 'S' or maze_neigh[1] == 'E':
        if curr_node.backtrack != 'U':
            valid_moves.append('U')
            # print("found U")

    if maze_neigh[4] == ' ' or maze_neigh[4] == 'S' or maze_neigh[4] == 'E':
        if curr_node.backtrack != 'L':
            valid_moves.append('L')
            # print("found U")


    if maze_neigh[6] == ' ' or maze_neigh[6] == 'S' or maze_neigh[6] == 'E':
        if curr_node.backtrack != 'R':
            valid_moves.append('R')
            # print("found U")


    if maze_neigh[9] == ' ' or maze_neigh[9] == 'S' or maze_neigh[9] == 'E':
        if curr_node.backtrack != 'D':
            valid_moves.append('D')
            # print("found U")

    if len(valid_moves) == 0:
        valid_moves.append(curr_node.backtrack)

    return valid_moves

# ----------------------------------------------------------------------------------------------------------

# Guards for 1st iteration, context request.
ready_for_input = False
need_context_update = True
mazeTreeStart = Node(name="", move="", backtrack="")
curr_node = mazeTreeStart
# Debug
counter = 0

# The maze game loop is contained in the maze.solve() method. Call the constructor, and then enter a for loop
#  to receive outputs from maze.solve(), and respond with our inputs.
maze = ASCIImaze.Maze("maze_map1", show_moves=False, show_coords=False)
for output in maze.solve():
    if (not ready_for_input):
        # On the first iteration, we get the maze to solve. Bring it into the LLM, then goto next iteration; that
        #  will be the first opportunity to pass an input to the ASCIImaze solver.
        # print(output)
        # Going to pull the maze into the LLM's chat history now.
        user_prompt=f"""```{output}```"""
        maze_to_solve = get_completion(extract_maze_system_prompt, user_prompt)
        ready_for_input = True
        continue

    if (need_context_update):
        # Grab context by passing '3' to ASCIIMaze. Isolated so can use stored context instead.
        # Must go to next iteration b/c only then will 'output' be updated with the 3x3 neighborhood.
        maze.get_user_move('3')
        need_context_update = False
        continue

    if "Maze solved!" in output:
        # Victory!
        print(output)
        break


    # Pull in the current 3x3 neighborhood, IF the current node does not already have a state. Otherwise, this is a
    #  backtracking situation.

    # Debug
    print(f"Output reads: {output}")
    if curr_node.name == "":
        # Case for a node not previously visited.
        user_prompt = f"```{output}```"
        neigh = get_completion(generate_thought_context_system_prompt, user_prompt)
        curr_node.name = neigh.content


    # Evaluate the current position; this is actually evaluating the through generated in the previous round.
    # Possible results are {{valid moves}, <just the backtrack direction>}
    # Backtrack if so indicated, generate a new thought if there are other valid moves.
    # The ASCIImaze.maze.solve() method, the main game loop, outputs a message when you correctly solve the maze, you
    #  don't need to figure out if the LLM has won the game. Just check for the victory message. Done above.


    print(f"Maze neighborhood is {curr_node.name}")

    valid_moves_cleaned = evaluate_thought(curr_node.name, curr_node)

# BUG TO SQUASH: valid_moves_cleaned should not include the backtrack direction, unless it is the only move left. Change evaluate_thought().

    print(f"Valid moves are {valid_moves_cleaned}")



    if len(valid_moves_cleaned) == 1 and valid_moves_cleaned[0] == curr_node.backtrack:
        # The only move detected is the backtrack direction, so follow it, both in the maze and the tree.
        maze.get_user_move(curr_node.backtrack)
        curr_node = tree_backtrack(curr_node, curr_node.backtrack)

        # Do not update context, the state in curr_node has already been updated. BUT this blows up an assumption :(
        need_context_update = False

        # The valid moves from the last eval step are no longer correct. Go to the next iteration, get a new move set.
        continue

    # Generate a thought
    # The LLM chooses a next move, which will be a child of the current node
    # Again, inner monologues are used to recreate the 2D mazes.
    user_prompt = f"""```{maze_to_solve.content}, {curr_node.name}```"""

    # Adding the backtrack move to thought generation. Remember to deal with root node (no parent).
    valid_moves_temp = ", ".join(valid_moves_cleaned)

    # Initial value for LLM_move, for loop entry
    LLM_move_cleaned = ['Z']

    # Loop to check for valid move.
    # LLM_move_cleaned should be a single-element list. If that element is not part of the valid move set, try again.
    while valid_moves_temp.find(LLM_move_cleaned[0]) < 0:

        print(f"Moves going to LLM are {valid_moves_temp}")

        if curr_node.parent != None:
            LLM_move = get_completion(generate_thought_system_prompt.format(vmove=valid_moves_temp,
                                                                    backmove=curr_node.backtrack), user_prompt)
        else:
            LLM_move = get_completion(generate_thought_system_prompt.format(vmove=valid_moves_temp,
                                                                        backmove=None), user_prompt)

        LLM_move_temp = re.sub(r'\&\&\&.+\&\&\&', '', LLM_move.content, flags=re.DOTALL)
        # Again, another regex needed.
        LLM_move_cleaned = re.findall("[A-Z]", LLM_move_temp)
        print(f"I choose move {LLM_move_cleaned[0]}")
    # Error check
    if len(LLM_move_cleaned) > 1:
        print("Extra stuff in the generated thought, aborting")
        break

    # Make the chosen move, and add the corresponding node to the tree.
    maze.get_user_move(LLM_move_cleaned[0])
    curr_node = add_node(curr_node, LLM_move_cleaned[0])
    if curr_node is None:
        print("You goofed.")
        print(LLM_move_cleaned)
    # Debug
    counter = counter + 1
    print(f"I choose move {LLM_move_cleaned}")
    print(f"This is move {counter}")

    # Need the next 3x3 neighborhood.
    need_context_update = True

print("I finished")
print("Token totals for maze:", TOKEN_TOTALS)
reset_token_totals()


Output reads: Here is a 3x3 of your current position:
###
 * 
 # 
Enter a move direction (U, D, L, R, or 3 (for a 3x3 of the current position)): 
Maze neighborhood is ###| * | # 
evaluate thought recevied ###| * | # 
Valid moves are ['L', 'R']
Moves going to LLM are L, R
I choose move R
I choose move ['R']
This is move 1
Output reads: Here is a 3x3 of your current position:
###
S*#
# #
Enter a move direction (U, D, L, R, or 3 (for a 3x3 of the current position)): 
Maze neighborhood is ###|S*#|# #
evaluate thought recevied ###|S*#|# #
Valid moves are ['D']
Moves going to LLM are D
I choose move D
I choose move ['D']
This is move 2
Output reads: Here is a 3x3 of your current position:
S #
#*#
# #
Enter a move direction (U, D, L, R, or 3 (for a 3x3 of the current position)): 
Maze neighborhood is S #|#*#|# #
evaluate thought recevied S #|#*#|# #
Valid moves are ['D']
Moves going to LLM are D
I choose move D
I choose move ['D']
This is move 3
Output reads: Here is a 3x3 of your current pos

# Report
## General Tree-of-Thought prompt
Tree-of-Thought (ToT) prompting serves as a way to guide an LLM to resolve issues through a "tree" of smaller steps rather than producing an answer in one go. First, we establish what constitutes one “thought” for this specific task (in our maze, a thought equals deciding which next move to take using the current 3x3 view). Next, we request the model to generate a candidate move, evaluate it (prefer forward openings while backtracking is avoided unless it's the only choice), and follow a search plan (we used DFS with backtracking). To enforce reliability of the model, our prompts are strict: (1) reformat the full maze to one line using |, (2) reformat the 3x3 to exactly 11 characters, (3) output only exactly one allowed move, avoid backtracking unless absolutely necessary, and follow a strict order of priorities for stable behaviour. More broadly, the ToT framework is employed in several problems within which the LLM continuously maintains a tree of candidate “thoughts” and performs self-evaluation while integrating classic search algorithms such as BFS/DFS with look-ahead and backtracking.

## Performance of LLM in solving mazes

| Maze | Solved (Y/N)|Moves|Prompt Tokens|Completion Tokens|Total Tokens|
|:----:|:-----------:|:---:|:-----------:|:---------------:|:----------:|
|Maze 1|      YES    |  15 |    12320    |       152       |   12472    |
|Maze 2|      YES    | 17  |    14033    |      173        |  14206     |
|Maze 3|      YES    |  7  |      5887   |    75           |  5962      |

We report for each maze the success, total moves, prompt tokens, completion tokens and total tokens. In general, the number of prompt tokens tends to be higher because each prompt contains the whole maze and the 3x3 context grid, while completion tokens remain smaller since the model outputs one letter for the move. A higher number of moves tends to be related with more backtracking and therefore more calls which leads to more tokens. After the formatting prompts and the avoid backtrack unless necessary rule were tightened, the agent's behavior became more predictable. This, in turn, reduces excess moves and ensures token use is more uniform per maze.

## Should LLMs be used for solving mazes
I suggest utilizing the LLM as an assistant rather than the primary problem solver. Allow traditional methods (DFS/BFS) to manage maze state, legality checks, automatic backtracking, and stopping rules. Limit the LLM's involvement to small tasks, like selecting a single move from a small valid set and resolving ties, with strict input/output (how we flattened the maze, 11-char 3X3 and output one letter). It's much easier to control costs in this scenario as longer prompts waste more tokens.

What we observed in the lab supports this. The LLM may meander, execute the same steps, or overlook basic spatial cues without constraints. Tighter formatting and the instruction “avoid backtracking unless necessary” helped but still left some issues. This supports the evidence that LLMs are spatially and structurally uneven unless controlled with specific tasks and constraints [1]. Prompting with text-based spatial tasks will not solve the problem. The best performance was achieved with methods that incorporate basic parsing or logic alongside search to define a new hybrid approach [2].

In conclusion, I suggest using LLMs as an LLM-assistant rather than an LLM-only solution. For small classroom mazes, our Tree-of-Thought loop with strict formatting is fine and cost-effective. For larger or reliability-critical problems, an algorithmic solver is preferable, letting the LLM provide lightweight heuristics or rankings, while keeping iteration caps and token tracking to manage cost [1][2].

## References
[1] Nathan Bos, “Language Models and Spatial Reasoning: What's Good, What Is Still Terrible, and What Is Improving,” Medium, 2024 https://medium.com/data-science/language-models-and-spatial-reasoning-whats-good-what-is-still-terrible-and-what-is-improving-175d2099eb4c

[2] Li, Hogg, Cohn, “Advancing Spatial Reasoning in Large Language Models: An In-Depth Evaluation and Enhancement Using the StepGame Benchmark,” AAAI-24

ECE 449 Laboratory #2: LLM Prompt Engineering PDF

Shunyu Yao, Dian Yu, Jeffrey Zhao, Izhak Shafran, Thomas L. Griffiths, Yuan Cao, Karthik Narasimhan, “Tree of Thoughts: Deliberate Problem Solving with Large Language Models,” https://arxiv.org/abs/2305.10601

https://platform.openai.com/docs/api-reference/introduction

https://www.promptingguide.ai/techniques/tot

https://cookbook.openai.com/examples/how_to_stream_completions#4-how-to-get-token-usage-data-for-streamed-chat-completion-response

https://www.reddit.com/r/ChatGPTCoding/comments/11zm880/how_do_i_get_token_usage_for_each_chatcompletion/